In [ ]:
import pandas as pd
import csv
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
def get_data(path): 
 list_row=[]
 date=[]
 traffic=[]
 with open(path,'r') as file:
  reader = csv.reader(file)
  for row in reader:
    if row[0]!='Date':
     traffic.append(row[1])
     date.append(row[0])     
     list_row.append(row)
 #print(date)
 #print(traffic)
 #print(len(traffic))
 return list_row,date,traffic

In [ ]:
list_row,date,traffic = get_data('/home/abh/Documents/Python/Untitled Folder/submission (1).csv')
print(date)
print(traffic)

In [ ]:
list_row[1460:]

In [ ]:
def calendar(date):
 d1=[]
 m1=[]
 y1=[]
 for i in date:
   a = i.split('/')
   if a[0] not in d1:
     d1.append(a[0])
   if a[1] not in m1:
     m1.append(a[1])
   if a[2] not in y1:
     y1.append(a[2])
 m2={'1':31,'2':28,'3':31,'4':30,'5':31,'6':30,'7':31,'8':31,'9':30,'10':31,'11':30,'12':31}
 date=[]
 for i in y1:
   for j in m2:
     len = m2[j]
     if (int(i)%4)==0 and j=='2':
       len=29
     for k in range(1,len+1):
       temp = str(k)+'/'+j+'/'+i
       date.append(temp)
 return date,y1,m1,d1

In [ ]:
date,year_all,month_all,date_all = calendar(date)
print(date)
print(year_all)
print(month_all)
print(date_all)

In [ ]:
def holiday_calendar():
 holiday=[]
 for i in year_all:
  a = "/15/8/" +i
  holiday.append(a)
  b = '26/1/'+i
  holiday.append(b) 
  b = '14/11/'+i
  holiday.append(b)
  b = '26/10/'+i
  holiday.append(b)
  b = '1/1/'+i
  holiday.append(b)
  b = '3/3/'+i
  holiday.append(b)
  b = '31/12/'+i
  holiday.append(b)
 return holiday

In [ ]:
holiday = holiday_calendar()
print(holiday)

In [ ]:
def date_to_day(list_row):
 week=['mon','tues','wed','thur','fri','sat','sun']
 count=3
 day={}
 for i in date:
   if count==7:
     count=0
   day[i]=week[count]
   count=count+1
 for row in list_row:
   if row[0] in holiday:
     row.append(1)
   else:
     row.append(0)
   row.append(day[row[0]])
 return day,list_row,week

In [ ]:
day,list_row,week = date_to_day(list_row)

In [ ]:
print(day)
print(list_row[0:10])

In [ ]:
def to_graph():
 listt=[]
 listd=[]
 for i in list_row:
    listt.append(int(i[1]))
    listd.append(i[0])
    ticks =  [0,365,365*2,365*3,365*4]
    labels = ['1/1/2015','1/1/2016','1/1/2017','1/1/2018','1/1/2019']
 return listt,listd,ticks,labels

In [ ]:
listt,listd,ticks,labels = to_graph()

In [ ]:
plt.plot(listd, listt,color='purple')
plt.xticks(ticks,labels,rotation=17)
plt.show()

In [ ]:
def max(list1):
  maximus=0
  for i in list1:
    if int(i)>maximus:
      maximus=int(i)
  return maximus

In [ ]:
max1= max(traffic)
print(max1)

In [ ]:
def pre_process(max,list_row):
 for i in range(0,len(list_row)):
   list_row[i][1] = int(list_row[i][1])/max
 return list_row

In [ ]:
list_row = pre_process(max1,list_row)
print(list_row[0:10])

In [ ]:
def number_to_one_hot(list1):
  listi=[]
  for i in range(0,len(list1)):
    listi.append(i)
  one_hot = keras.utils.to_categorical(listi)
  dictf={}
  count=0
  for i in listi:
    code = one_hot[i]
    ind = list1[count]
    dictf[ind] = code
    count = count+1
  return dictf

In [ ]:
#Defining Seasonality
season=['winter','summer','autumn','spring','rainy']
def cur_season(season,date):
  sess= number_to_one_hot(season)
  d = date.split('/')
  w = ['11','12','1']
  a=0
  s = ['5','6','7','8']
  sp = ['2','3','4']
  aut = ['10']
  ra = ['9']
  if d[1] in w:
    a= sess['winter']
  if d[1] in s:
    a= sess['summer']
  if d[1] in sp:
    a= sess['spring']
  if d[1] in aut:
    a= sess['autumn']
  if d[1] in ra:
    a= sess['rainy']
  return a

In [ ]:
#Encoding Dates
def date_enc(date):
  days=[]
  months=[]
  years=[]
  final={}
  count=0
  for i in date:
    datess = i.split("/")
    if int(datess[0]) not in days:
      days.append(int(datess[0]))
    if int(datess[1]) not in months:
      months.append(int(datess[1]))
    if int(datess[2]) not in years:
      years.append(int(datess[2]))
  day_enc = number_to_one_hot(days)
  month_enc = number_to_one_hot(months)
  count=0
  #print(days,months,years)
  years_enc = number_to_one_hot(years)
  #print("year_enc=",years_enc)
  count=0
  #print(years)
  #print(len(dates_enc),len(months_enc),years_enc)
  return day_enc,month_enc,years_enc

In [ ]:
days,months,years = date_enc(date)
print(days)
print(months)
print(years)

In [ ]:
def date_to_enc(date,days,months,years):
  dates = date.split("/")
  day = days[int(dates[0])]
  month = months[int(dates[1])]
  year = years[int(dates[2])]
  return day,month,year

In [ ]:
#Converting Data into inputs

def conversion(week,days,months,years,list_row):
  inp_day = []
  inp_mon = []
  inp_year = []
  inp_week=[]
  inp_hol=[]
  out = []
  week1 = number_to_one_hot(week)
  count=0
  for row in list_row:
       #print(row[0])
       d = row[0]
       d_split=d.split('/')
       if d_split[2]==str(year_all[0]) or d_split[2]==str(year_all[-1]):
         continue
       #print(d)
       d1,m1,y1 = date_to_enc(d,days,months,years)
       inp_day.append(d1)
       inp_mon.append(m1)
       inp_year.append(y1)
       week2 = week1[row[3]]
       inp_week.append(week2)
       t1 = row[1]
       inp_hol.append([row[2]])
       #for i in range(0,len(o)):
       #  b[i+len(t)] = t[i]
       out.append(t1)
       count = count+1
  return inp_day,inp_mon,inp_year,inp_week,inp_hol,out

In [ ]:
inp_day,inp_mon,inp_year,inp_week,inp_hol,out = conversion(week,days,months,years,list_row)
inp_day = np.array(inp_day)
inp_mon = np.array(inp_mon)
inp_year = np.array(inp_year)
inp_week = np.array(inp_week)
inp_hol = np.array(inp_hol)
#inp = inp1.reshape((inp1.shape[0],1,inp1.shape[1]))
#inp = inp.reshape((inp.shape[0],1,inp.shape[1]))
print(inp_day.shape)
print(inp_day.dtype)
print(inp_mon.shape)
print(inp_mon.dtype)
print(inp_year.shape)
print(inp_year.dtype)
print(inp_week.shape)
print(inp_week.dtype)
print(inp_hol.shape)
print(inp_hol.dtype)
out = np.array(out)
print(out.shape)
print(out.dtype)

In [ ]:
def other_inputs(season):
 inp7=[]
 inp_prev=[]
 inp_sess=[]
 count=0
 for row in list_row:
  ind = count
  count=count+1
  d = row[0]
  d_split=d.split('/')
  if d_split[2]==str(year_all[0]) or d_split[2]==str(year_all[-1]):
    continue
  sess = cur_season(season,d)
  inp_sess.append(sess)
  #print(d_year)
  t7=[]
  t_prev=[]
  t_prev.append(list_row[ind-365][1])
  for j in range(0,7):
       t7.append(list_row[ind-j-1][1])
  inp7.append(t7)
  inp_prev.append(t_prev)
 return inp7,inp_prev,inp_sess

In [ ]:
inp7,inp_prev,inp_sess = other_inputs(season)
inp7 = np.array(inp7)
inp7= inp7.reshape(inp7.shape[0],inp7.shape[1],1)
print(inp7.shape)
inp_prev = np.array(inp_prev)
print(inp_prev.shape)
inp_sess = np.array(inp_sess)
print(inp_sess.shape)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense,LSTM,Flatten
from tensorflow.keras.layers import concatenate

input_day = Input(shape=(inp_day.shape[1],),name = 'input_day')
input_mon = Input(shape=(inp_mon.shape[1],),name = 'input_mon')
input_year = Input(shape=(inp_year.shape[1],),name = 'input_year')
input_week = Input(shape=(inp_week.shape[1],),name = 'input_week')
input_hol = Input(shape=(inp_hol.shape[1],),name = 'input_hol')
input_day7 = Input(shape=(inp7.shape[1],inp7.shape[2]),name = 'input_day7')
input_day_prev = Input(shape=(inp_prev.shape[1],),name = 'input_day_prev')
input_day_sess = Input(shape=(inp_sess.shape[1],),name = 'input_day_sess')

x1 = Dense(5, activation='relu')(input_day)
x2 = Dense(5, activation='relu')(input_mon)
x3 = Dense(5, activation='relu')(input_year)
x5 = Dense(5, activation='relu')(input_week)
x6 = Dense(5, activation='relu')(input_hol)
x_10 = Dense(5, activation='relu')(input_day7)
x__10 = LSTM(5,return_sequences=True)(x_10)
x10 = Flatten()(x__10)
x11 = Dense(5, activation='relu')(input_day_prev)
x12 = Dense(5, activation='relu')(input_day_sess)
c = concatenate([x1, x2,x3,x5,x6,x10,x11,x12])
layer1 = Dense(64,activation='relu')(c)
outputs = Dense(1, activation='sigmoid')(layer1)

model = Model(inputs=[input_day,input_mon,input_year,input_week,input_hol,input_day7,input_day_prev,input_day_sess], outputs=outputs)
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss=['mean_squared_error'],
              optimizer = 'adam',
              metrics = ['acc']
              )

In [ ]:
history = model.fit(
           x = [inp_day,inp_mon,inp_year,inp_week,inp_hol,inp7,inp_prev,inp_sess],
           y = out,
           batch_size=16,
           steps_per_epoch=50,
           epochs = 15,
           verbose=1,
           shuffle =False
           )

In [ ]:
def testing(date,traffic,order,maxj):
       out=[]
       d1,d2,d3 = date_to_enc(date,days,months,years)
       #print(da)
       #print(d[0])
       print('date=',date)
       #print(inp[0][count])
       #print(count)
       #t1 = int(traffic/maxj)
       d1 = np.array([d1])
       #print(d1.shape)
       d2 = np.array([d2])
       d3 = np.array([d3])
       #print(d2.shape)
       #print(d3.shape)
       #print(us.shape)
       week1 = number_to_one_hot(week)
       week2 = week1[day[date]]
       #print(day[date])
       week2=np.array([week2])
       #print(week2)
       #print("date=",date)
       if date in holiday:
         h=1
         print('holiday')
       else:
         h=0
         print("no holiday")
       h = np.array([h])
       #print(h)
       count=-1
       ind=0
       #print(date)
       #print(us)
       for i in list_row:
         count =count+1
         #print(count)
         #print(i[0],i[1])
         #print(date,us)
         if i[0]==date:
           #print("found")
           ind = count
       print("ind=",ind)
       sess = cur_season(season,date)
       #print(sess)
       #print(d_prev)
       t7=[]
       t_prev=[]
       t_prev.append(list_row[ind-365][1])
       print(list_row[ind-365])
       #print(list_row[ind-366*len(url)])
       #print(list_row[ind-365*len(url)])
       print("t-prev",t_prev)
       for j in range(0,7):
            t7.append(list_row[ind-j][1])
       t7 = np.array([t7])
       t7 = t7.reshape(1,7,1)
       print('t7=',t7)
       t_prev = np.array([t_prev])
       #print("prev=",t_prev)
       sess = np.array([sess])
       y_out = model.predict([d1,d2,d3,week2,h,t7,t_prev,sess])
       #print("output=",y_out)
       #print(y_out.shape)
       print(list_row[ind])
       print("output after processing=",y_out[0][0]*maxj)
       #sales on the date
       print("sales on date=",list_row[ind][1]*maxj)
       e1 = y_out[0][0]*maxj- list_row[ind][1]*maxj
       print('change from expected value= ',e1)
       return y_out[0][0]*maxj

In [ ]:
a= testing('31/12/2019',116,50,max1)

In [ ]:
print(list_row)

In [ ]:
print(date)

In [ ]:
def testmore(date_all,maxj):
  y=[]
  d=[]
  for i in range(1461,1826):
    print(date_all[i])
    y1 = testing(date_all[i],100,50,maxj)
    y.append(y1)
    d.append(date_all[i])
  return y,d

In [ ]:
y,d = testmore(date,max1)

In [ ]:
plt.plot(d,y,color='red')
plt.title('predicted')
plt.xlabel("Date")
plt.ylabel("Sales")
plt.show()
plt.plot(listd[1460:],listt[1460:],color='purple')
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title('actual values')
plt.show()

In [ ]:
plt.plot(d,y,color='red',label='predicted')
plt.plot(listd[1460:],listt[1460:],color='purple',label="actual")
plt.xlabel("Date")
plt.ylabel("Sales")
leg = plt.legend()
plt.show()